In [500]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from gensim.models import Word2Vec
import random
from random import sample
from collections import Counter, defaultdict
from itertools import chain

# 기본추천

- 사용자의 플레이리스트 기반. 플레이리스트에 등장하는 가수들과 유사한 가수들 자동 랜덤추천
- 비율은 3 대 7
> 가수로 추천하는것????
> 생각해보니 ost 등등 가수는 전혀 관련 없는 경우도 있음.    
> 곡기반 기본추천도 생각해야겠음.   
> 협업필터링 방식의 추천이 더 나을 것 같다  

1. 플레이리스트별 가수 리스트 만들기.
2. 가수들 word2vec 모델 만들기.
3. 유사가수 추출.
4. 유사가수의 곡을 인기도 순으로 n곡씩 컷. 랜덤셔플

In [113]:
#플레이리스트별 가수 리스트로 만들기. ( 추가해서 저장 해놓을까 )
train = pd.read_json('train.json')
song_meta = pd.read_json('song_meta.json')

In [114]:
train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


In [115]:
song_meta.head(3)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2


In [118]:
# 노래 인기도
all_songs = list(chain(*train['songs']))
song_counts = dict(Counter(all_songs))
song_meta['popularity'] = song_meta['id'].apply(lambda x : song_counts.get(x) if song_counts.get(x) else 0)

In [121]:
# 각 플레이리스트에 포함된 노래를 부른 가수들을 가져오기 위한 함수
def get_artist(playlist):
    artists = set()
    for song in playlist:
        artists.update(songid_to_artists[song])
    return list(artists)

In [122]:
train['artists'] = train['songs'].progress_apply(get_artist)

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:04<00:00, 23478.17it/s]


In [123]:
train['artists']

0         [Tomi Swick, Christophe Mae, Honeymoon Suite, ...
1         [서태지, 이승철, Tsukemen, 이승환, 브로콜리너마저, Above Envy,...
2         [빌런 (Villain), 달 좋은 밤, 소심한 오빠들, 김새한길, 샘김 (Sam ...
3         [스윗소로우 (SWEET SORROW), 정준일, 멜로망스, 토이, 이민혁, Kri...
4         [쿨 (COOL), 김현정, 신화, 유피, S.E.S., 샤크라, 이정현, 지누션,...
                                ...                        
115066    [Deftones, Deicide, Dream Theater, Arch Enemy,...
115067    [Parachute Youth, TheElement, Clyde Trevor, Ko...
115068    [씨야, 채동하, FTISLAND (FT아일랜드), 벤, 김창기, 성시경, SG 워...
115069    [Angus & Julia Stone, Daniel Hart, Perfume Gen...
115070    [Angus & Julia Stone, Laura Jansen, Sia, Sky F...
Name: artists, Length: 115071, dtype: object

In [124]:
## word2vec 필요
## 10분 정도 걸림
# w2v_model = Word2Vec(train['artists'].tolist(), min_count = 1, size = 100 , window = 200, sg = 1)
# w2v_model.save('artist_w2v.model')

# 훈련된 w2v 모델 불러오기
w2v_model = Word2Vec.load('../artist_w2v.model')

---

In [355]:
#샘플유저 뽑기
sample_user = train.sample(1)
sample_user

,tags,id,plylst_title,songs,like_cnt,updt_date,artists
97670,"[감성, 운동, 일렉팝, 감각적인, 스포츠, 걷기, 모델워킹, 일렉, 팝, 워킹]",5443,모델워킹을 부르는 몽환적인 일렉팝,"[289177, 47436, 509417, 196342, 51613, 161380,...",16,2019-03-29 15:45:33.000,"[Terror Jr, DUSK, Matoma, Oliver Nelson, Kygo,..."


In [356]:
# 샘플유저 아티스트리스트 가져오기
user_random_artist_list = random.sample(*sample_user['artists'].values, 3)

In [357]:
user_random_artist_list

['Oliver Nelson', 'Christopher', 'Clairo']

In [358]:
# 유사 가수 찾기 - 각 가수 따로 계산
user_recommend_artists = []
for artist in user_random_artist_list:
    temp = w2v_model.wv.most_similar(artist, topn = 5)
    user_recommend_artists.extend([x for x , _ in temp])
user_recommend_artists = sample(user_recommend_artists, 5)
user_recommend_artists

['The Knocks', 'Captain Cuts', 'Tobtok', 'gnash', 'Jake Miller']

In [367]:
# 유사 가수 찾기 - 벡터의 평균으로 계산
user_recommend_artists = []

temp = w2v_model.wv.most_similar(user_random_artist_list, topn = 10)
user_recommend_artists.extend([x for x , _ in temp])
user_recommend_artists = sample(user_recommend_artists, 5)
user_recommend_artists


['The Knocks', 'SG Lewis', 'Blue Lotus', 'Gryffin', 'Terror Jr']

 user_random_artist_list 에서 곡을 가져오고 - 기존에 있던 곡은 제외한다.  
 user_recommend_artists 에서도 곡을 가져오고

In [177]:
# 기존 가수 노래 가져오기
artist_songs = []
for artist_ in user_random_artist_list:
    temp_songs = song_meta[song_meta['artist_name_basket'].apply(lambda x : any(artist_ in artist for artist in x))]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    
    artist_songs.extend(temp_songs['id'])

In [178]:
len(artist_songs)

30

In [179]:
# 이미 있는 노래는 제거한다.
artist_songs =  set(artist_songs) - set(*sample_user['songs'])

In [180]:
# 추천된 가수의 노래 가져오기.
new_artist_songs = []
for artist_ in user_recommend_artists:
    temp_songs = song_meta[song_meta['artist_name_basket'].apply(lambda x : any(artist_ in artist for artist in x))]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    
    new_artist_songs.extend(temp_songs['id'])

In [181]:
len(new_artist_songs)

50

3대 7 비율로 섞기.   
9곡 21 곡

In [182]:
final_recommendation = []
final_recommendation.extend(sample(artist_songs, int(30*0.3)))
final_recommendation.extend(sample(new_artist_songs, math.ceil(30* (1-0.3) )))

In [183]:
song_meta.iloc[final_recommendation][['song_name','artist_name_basket']]

,song_name,artist_name_basket
105451,어제처럼 굿나잇,[러블리즈]
358847,좋아한다고 해,[한살차이]
318184,Love You,[한살차이]
278810,키스미,[참깨와 솜사탕]
220954,작별하나,[러블리즈]
136428,종소리,[러블리즈]
622984,Candy Jelly Love,[러블리즈]
232972,이즐께,[참깨와 솜사탕]
578474,Ah-Choo,[러블리즈]
43689,혼자,[커피소년]


# 속도 개선필요
- 가수가 부른 노래를 찾을 때 좀 빨리 찾아보자.

In [497]:
# 가수 - 노래 테이블 참조해서 해보기
artist_flat = pd.DataFrame([[i, x] for i, y in song_meta['artist_name_basket'].iteritems() for x in y], columns=['i','artist'])
artist_flat = artist_flat.set_index('i')

In [498]:
artist_flat

,artist
i,
0,Various Artists
1,Murray Perahia
2,Peter Gabriel
3,Matoma
4,Jude Law
...,...
707984,Fela Kuti
707985,Cyndi Lauper
707986,윤종신


In [232]:
# 기존 가수 노래 가져오기
artist_songs = []
for artist_ in user_random_artist_list:
    temp_songs = song_meta.iloc[artist_flat[artist_flat['artist'] == artist_].index]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    
    artist_songs.extend(temp_songs['id'])

In [233]:
# 이미 있는 노래는 제거한다.
artist_songs =  set(artist_songs) - set(*sample_user['songs'])

In [234]:
# 추천된 가수의 노래 가져오기.
new_artist_songs = []
for artist_ in user_recommend_artists:
    temp_songs = song_meta.iloc[artist_flat[artist_flat['artist'] == artist_].index]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    
    new_artist_songs.extend(temp_songs['id'])

In [235]:
final_recommendation = []
final_recommendation.extend(sample(artist_songs, int(30*0.3)))
final_recommendation.extend(sample(new_artist_songs, math.ceil(30* (1-0.3) )))

In [236]:
song_meta.iloc[final_recommendation][['song_name','artist_name_basket']]

,song_name,artist_name_basket
334193,One Last Time,[Ariana Grande]
81523,7 rings,[Ariana Grande]
482423,Side To Side (Feat. Nicki Minaj),[Ariana Grande]
105599,Sorry,[Justin Guarini]
599783,Get Here,[Justin Guarini]
327339,End Of The Road,[Boyz II Men]
656247,One Heart Too Many,[Justin Guarini]
542127,Bang Bang,"[Jessie J, Ariana Grande, Nicki Minaj]"
678161,"thank u, next",[Ariana Grande]
153940,Dusk Till Dawn,"[ZAYN, Sia]"


# 정리

### 필요 테이블
- song_meta에 popularity 추가.
- train에 등장 가수 리스트 추가.
- artist_flat 노래 - 가수 테이블 (관계형 db 노래 가수 릴레이션쉽에 쓰면될듯)

### 가수를 선별하는 방법
- 플레이리스트 내 가수를 전부 사용 - *가수가 너무 많은경우 작동 안하는 경우 있음*
- 랜덤샘플링 해서 사용

### 선별된 가수와 유사한 추천가수를 가져오는 방법.
- 가수 벡터 전체를 입력.
- 가수 한명씩 입력하여 top5를 구하고 합친뒤 랜덤샘플링

In [445]:
%%time
#setting
rec_songs_cnt = 30  #최대 출력 곡 수.
rate_of_familiar_songs = 0.3  #아는 가수 노래 비율
artist_sample = 5 #아는 가수 얼마나 뽑을지.

song_cnt_from_fam = int(rec_songs_cnt * rate_of_familiar_songs)
song_cnt_from_rec = int(rec_songs_cnt * (1-rate_of_familiar_songs))

artist_songs = []
new_artist_songs = []
final_recommendation = []
user_recommend_artists = []

sample_user = train.sample(1)
print('샘플 유저 : \n')
display(sample_user)

#유저 아티스트 랜덤 샘플 가져오기
if len(*sample_user['artists'].values) > artist_sample:
    user_random_artist_list = random.sample(*sample_user['artists'].values, artist_sample)
else:
    user_random_artist_list = list(*sample_user['artists'].values)

print('기존 가수들 샘플 : \n', user_random_artist_list)


#유사 가수 찾기 (전체로 찾기)
temp = w2v_model.wv.most_similar(user_random_artist_list, topn = 7)
user_recommend_artists.extend([x for x , _ in temp])
# user_recommend_artists = sample(user_recommend_artists, 5)
print('선별 유사 가수들 : \n', user_recommend_artists)


# 기존 가수 노래 가져오기
for artist_ in user_random_artist_list:
    temp_songs = song_meta.iloc[artist_flat[artist_flat['artist'] == artist_].index]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    artist_songs.extend(temp_songs['id'])

# 새 가수 노래 가져오기    
for artist_ in user_recommend_artists:
    temp_songs = song_meta.iloc[artist_flat[artist_flat['artist'] == artist_].index]\
                                                           .sort_values(by= 'popularity', ascending = False)[:10]
    new_artist_songs.extend(temp_songs['id'])

# 이미 있는 노래는 제거한다.
artist_songs =  set(artist_songs) - set(*sample_user['songs'])

#노래 샘플로 뽑기
if len(artist_songs) >= song_cnt_from_fam:
    final_recommendation.extend(sample(artist_songs,  song_cnt_from_fam   ))
else:
    final_recommendation.extend(artist_songs)
    
if len(new_artist_songs) >= song_cnt_from_rec:
    final_recommendation.extend(sample(new_artist_songs,  song_cnt_from_rec    ))
else:
    final_recommendation.extend(new_artist_songs)

print('\n추천 리스트')
display(song_meta.iloc[final_recommendation][['song_name','artist_name_basket']])
print(f'곡 수 : {len(final_recommendation)}')

샘플 유저 : 



,tags,id,plylst_title,songs,like_cnt,updt_date,artists
43274,"[electronica, 오피셜차트, 영국, Pop, Rock]",57911,오피셜 싱글 차트 Top 100 2018년 6월 8일 ~ 14일,"[529616, 135193, 493762, 429377, 505231, 52953...",4,2018-11-19 15:07:05.000,"[Swae Lee, Sia, The Greatest Showman Ensemble,..."


기존 가수들 샘플 : 
 ['Liam Payne', 'Maren Morris', 'BlocBoy JB', 'Paloma Faith', 'Michelle Williams']
선별 유사 가수들 : 
 ['Ella Eyre', 'Grey', 'Jonas Blue', 'Karen Gibson', 'The Kingdom Choir', 'Dua Lipa', 'Camila Cabello']

추천 리스트


,song_name,artist_name_basket
334046,Shoot,[BlocBoy JB]
677277,Stack It Up (Feat. A Boogie Wit da Hoodie),[Liam Payne]
673855,Stargazer,[Paloma Faith]
578101,Bedroom Floor,[Liam Payne]
662611,Can`t Rely On You,[Paloma Faith]
135950,Strip That Down (Feat. Quavo),[Liam Payne]
94362,The Bones,[Maren Morris]
385896,Lullaby,"[Sigala, Paloma Faith]"
536335,For You (Fifty Shades Freed),"[Liam Payne, Rita Ora]"
136358,Room For 2,[Dua Lipa]


곡 수 : 30
Wall time: 1.37 s


# 여전히 느림. 개선 필요

# 가수가 부른 곡 딕셔너리 만들어놓기

In [517]:
artist_flat.reset_index(drop = False, inplace = True)

In [518]:
artist_flat

,i,artist
0,0,Various Artists
1,1,Murray Perahia
2,2,Peter Gabriel
3,3,Matoma
4,4,Jude Law
...,...,...
779480,707984,Fela Kuti
779481,707985,Cyndi Lauper
779482,707986,윤종신
779483,707987,Nature Piano


In [536]:
singer_songs = defaultdict(list)

In [537]:
artist_flat.progress_apply(lambda x : singer_songs[x['artist']].append(x['i']), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 779485/779485 [00:10<00:00, 73298.25it/s]


0         None
1         None
2         None
3         None
4         None
          ... 
779480    None
779481    None
779482    None
779483    None
779484    None
Length: 779485, dtype: object

## 다시해보기

In [543]:
%%time
#setting
rec_songs_cnt = 30  #최대 출력 곡 수.
rate_of_familiar_songs = 0.3  #아는 가수 노래 비율
artist_sample = 5 #아는 가수 얼마나 뽑을지.

song_cnt_from_fam = int(rec_songs_cnt * rate_of_familiar_songs)
song_cnt_from_rec = int(rec_songs_cnt * (1-rate_of_familiar_songs))

artist_songs = []
new_artist_songs = []
final_recommendation = []
user_recommend_artists = []

sample_user = train.sample(1)
print('샘플 유저 : \n')
display(sample_user)

#유저 아티스트 랜덤 샘플 가져오기
if len(*sample_user['artists'].values) > artist_sample:
    user_random_artist_list = random.sample(*sample_user['artists'].values, artist_sample)
else:
    user_random_artist_list = list(*sample_user['artists'].values)

print('기존 가수들 샘플 : \n', user_random_artist_list)


#유사 가수 찾기 (전체로 찾기)
temp = w2v_model.wv.most_similar(user_random_artist_list, topn = 7)
user_recommend_artists.extend([x for x , _ in temp])
# user_recommend_artists = sample(user_recommend_artists, 5)
print('선별 유사 가수들 : \n', user_recommend_artists)


# 기존 가수 노래 가져오기
for artist_ in user_random_artist_list:
    temp_songs = song_meta.iloc[singer_songs[artist_]].sort_values(by= 'popularity', ascending = False)[:10]
    artist_songs.extend(temp_songs['id'])

# 새 가수 노래 가져오기    
for artist_ in user_recommend_artists:
    temp_songs = song_meta.iloc[singer_songs[artist_]].sort_values(by= 'popularity', ascending = False)[:10]
    new_artist_songs.extend(temp_songs['id'])

# 이미 있는 노래는 제거한다.
artist_songs =  set(artist_songs) - set(*sample_user['songs'])

#노래 샘플로 뽑기
if len(artist_songs) >= song_cnt_from_fam:
    final_recommendation.extend(sample(artist_songs,  song_cnt_from_fam   ))
else:
    final_recommendation.extend(artist_songs)
    
if len(new_artist_songs) >= song_cnt_from_rec:
    final_recommendation.extend(sample(new_artist_songs,  song_cnt_from_rec    ))
else:
    final_recommendation.extend(new_artist_songs)

print('\n추천 리스트')
display(song_meta.iloc[final_recommendation][['song_name','artist_name_basket']])
print(f'곡 수 : {len(final_recommendation)}')

샘플 유저 : 



,tags,id,plylst_title,songs,like_cnt,updt_date,artists
53127,"[감성, 비, 록, 흐림, Rock, 락, 인디]",10671,감성 충만한 일본의 인디 록~♪,"[73698, 494037, 534974, 204900, 480631, 529987...",5,2018-04-04 10:51:59.000,"[Ellegarden, Barbars, Mamadrive, Yoru No Honki..."


기존 가수들 샘플 : 
 ['RADWIMPS (래드윔프스)', 'Bo-Peep', 'NEENEE(니니)', 'ariel makes gloomy', 'Ellegarden']
선별 유사 가수들 : 
 ['코도모드래곤 (Codomo Dragon)', 'The MANJI', 'Karasu', 'The Pees', 'Yoru No Honki Dance', 'TERRA ROSA', 'Jay Walk ']

추천 리스트


,song_name,artist_name_basket
201980,infinite refrain,[ariel makes gloomy]
521852,Let`s Go Bananas,[Bo-Peep]
202150,三葉のテ-マ / Theme of Mitsuha,[RADWIMPS (래드윔프스)]
78306,"Marry Me (CF- 삼성 애니콜 3.5 `정일우, 고아라`편)",[Ellegarden]
480477,Rise Up,[Bo-Peep]
195470,Make A Wish,[Ellegarden]
549355,Thank You,[Bo-Peep]
172408,夢燈籠 / Dream lantern,[RADWIMPS (래드윔프스)]
505319,Boring Life,[Bo-Peep]
294044,體にやさしいパンク / Karadani Yasasii Funk (몸에 좋은 FUNK),[The Pees]


곡 수 : 30
Wall time: 38.9 ms


# ms 단위로 향상됨
# 미리 딕셔너리를 만들어 놔야 빨라짐.